# 1. SEP28 PREPARATION

In [1]:
!unzip sep28_all_together

unzip:  cannot find or open sep28_all_together, sep28_all_together.zip or sep28_all_together.ZIP.


In [2]:
!ls ./sep28_all_together/

StutterTalk_43_55.wav	     WomenWhoStutter_88_92.wav
StutterTalk_43_56.wav	     WomenWhoStutter_88_93.wav
StutterTalk_43_57.wav	     WomenWhoStutter_88_94.wav
StutterTalk_43_58.wav	     WomenWhoStutter_88_95.wav
StutterTalk_43_59.wav	     WomenWhoStutter_88_96.wav
StutterTalk_4_35.wav	     WomenWhoStutter_88_97.wav
StutterTalk_43_5.wav	     WomenWhoStutter_88_98.wav
StutterTalk_43_60.wav	     WomenWhoStutter_88_99.wav
StutterTalk_43_61.wav	     WomenWhoStutter_88_9.wav
StutterTalk_43_62.wav	     WomenWhoStutter_8_8.wav
StutterTalk_43_63.wav	     WomenWhoStutter_89_0.wav
StutterTalk_43_64.wav	     WomenWhoStutter_89_10.wav
StutterTalk_43_65.wav	     WomenWhoStutter_89_11.wav
StutterTalk_43_66.wav	     WomenWhoStutter_89_12.wav
StutterTalk_43_67.wav	     WomenWhoStutter_89_13.wav
StutterTalk_43_68.wav	     WomenWhoStutter_89_14.wav
StutterTalk_43_69.wav	     WomenWhoStutter_89_15.wav
StutterTalk_4_36.wav	     WomenWhoStutter_89_16.wav
StutterTalk_43_6.wav	     WomenWhoStutter_89_17.wav
S

In [3]:
import os

root_folder = './sep28_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 21856


In [4]:
import pandas as pd

df_sep = pd.read_csv('SEP_28k_labels.csv')
df_sep.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [ ]:
print(df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep['EpId'] == 99) & (df_sep['ClipId'] == 99)]['NoStutteredWords'])

27324    2
Name: NoStutteredWords, dtype: int64


In [6]:
tmp = df_sep[(df_sep['NoStutteredWords'] == 3) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] == 0)
                        & (df_sep['Block'] == 0)
                        & (df_sep['SoundRep'] == 0)    
                        & (df_sep['WordRep'] == 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0)]

len(df_sep), len(tmp)

(28177, 2479)

In [27]:
col_list_nostutter = [ 'Unsure', 
            'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'WordRep',
            'DifficultToUnderstand',
            'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech']

tmp = df_sep[(df_sep['NoStutteredWords'] == 3) & (df_sep[col_list_nostutter].sum(axis=1) == 0)]

len(df_sep), len(tmp)

(28177, 2479)

### Creating no stutter files with confidence 3 and count of 800

In [ ]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_balanced'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[8] == 3 and sum(features) == 3:
        print(show_name,pod_id,clip_id, features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)]['NoStutteredWords'])
        count += 1
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
    if count == 800:
        break


print(count)

WomenWhoStutter 88 119 [0 0 0 0 0 0 0 0 3 0 0 0] 26401    3
Name: NoStutteredWords, dtype: int64
StutterTalk 25 11 [0 0 0 0 0 0 0 0 3 0 0 0] 11288    3
Name: NoStutteredWords, dtype: int64
MyStutteringLife 30 166 [0 0 0 0 0 0 0 0 3 0 0 0] 6950    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 93 24 [0 0 0 0 0 0 0 0 3 0 0 0] 26692    3
Name: NoStutteredWords, dtype: int64
StutterTalk 41 38 [0 0 0 0 0 0 0 0 3 0 0 0] 12575    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 23 16 [0 0 0 0 0 0 0 0 3 0 0 0] 21359    3
Name: NoStutteredWords, dtype: int64
StutterTalk 40 34 [0 0 0 0 0 0 0 0 3 0 0 0] 12531    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 2 178 [0 0 0 0 0 0 0 0 3 0 0 0] 19669    3
Name: NoStutteredWords, dtype: int64
HeStutters 0 2 [0 0 0 0 0 0 0 0 3 0 0 0] 2    3
Name: NoStutteredWords, dtype: int64
StutterTalk 31 30 [0 0 0 0 0 0 0 0 3 0 0 0] 11851    3
Name: NoStutteredWords, dtype: int64
MyStutteringLife 23 43 [0 0 0 0 0 0 0 0 3 0 0 0] 6407    3
Name: 

In [7]:
import os

root_folder = './sep28_balanced/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 1636


### Creating stutter files for  Prolongation, SoundRep, WordRep to be detected as stuttering situation and count of 836

In [15]:
# tmp = df_sep[(df_sep['NoStutteredWords'] == 0) 
#                         & (df_sep['Unsure'] == 0)
#                         & (df_sep['PoorAudioQuality'] == 0)
#                         # & (df_sep['Prolongation'] == 0)
#                         & (df_sep['Block'] == 0)
#                         & ((df_sep['SoundRep'] >= 2) | (df_sep['Interjection'] >= 2) | ((df_sep['Prolongation'] == 0))    
#                         # & (df_sep['WordRep'] >= 2)
#                         & (df_sep['DifficultToUnderstand'] == 0)
#                         # & (df_sep['Interjection'] == 0)
#                         & (df_sep['NaturalPause'] == 0)
#                         & (df_sep['Music'] == 0)
#                         & (df_sep['NoSpeech'] == 0)]

tmp = df_sep[(df_sep['NoStutteredWords'] == 0) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] == 0)
                        & (df_sep['Block'] >= 0)
                        & ((df_sep['SoundRep'] == 0))    
                        & (df_sep['WordRep'] == 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0)]

len(df_sep), len(tmp)

(28177, 84)

In [ ]:
col_list_stutter = [ 'Unsure', 
            'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'WordRep',
            'DifficultToUnderstand',
            'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech']

tmp = df_sep[(df_sep['NoStutteredWords'] == 0) & (df_sep[col_list_stutter].sum(axis=1) == 3)]

len(df_sep), len(tmp)

(28177, 1295)

In [ ]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_balanced'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if lst[i] != 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [2, 4, 5]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id)
        print(features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)][['Prolongation','SoundRep','WordRep','Interjection']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
    if count == 900:
        break


print(count)

[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  SoundRep  WordRep  Interjection
26740             0         0        3             0
HeStutters 21 214
[0 0 0 0 3 0 0 0 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
3255             0         3        0             0
WomenWhoStutter 2 162
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  SoundRep  WordRep  Interjection
19653             0         0        3             0
StutterTalk 68 106
[0 0 2 0 1 0 0 0 0 0 0 0]        Prolongation  SoundRep  WordRep  Interjection
14206             2         1        0             0
WomenWhoStutter 97 156
[0 0 0 0 2 1 0 0 0 0 0 0]        Prolongation  SoundRep  WordRep  Interjection
27140             0         2        1             0
StutterTalk 8 169
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  SoundRep  WordRep  Interjection
10617             0         0        3             0
WomenWhoStutter 42 67
[0 0 2 0 1 0 0 0 0 0 0 0]        Prolongation  SoundRep  WordRep  Interjection
22701    

In [ ]:
import os

root_folder = './sep28_balanced/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)


Total number of files: 1636


### Renaming files in sep28_balanced to nostutter (800) and stutter files(836):

In [ ]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_balanced/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] == 3:
        print(features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    else:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0

In [4]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_balanced/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 800 , 836 and tot 1636


In [ ]:
!ls ./sep28_balanced/

nostutter_HeStutters_0_12.wav	       stutter_HeStutters_12_42.wav
nostutter_HeStutters_0_20.wav	       stutter_HeStutters_1_244.wav
nostutter_HeStutters_0_2.wav	       stutter_HeStutters_1_246.wav
nostutter_HeStutters_0_32.wav	       stutter_HeStutters_12_56.wav
nostutter_HeStutters_0_35.wav	       stutter_HeStutters_1_258.wav
nostutter_HeStutters_0_4.wav	       stutter_HeStutters_12_70.wav
nostutter_HeStutters_0_6.wav	       stutter_HeStutters_12_74.wav
nostutter_HeStutters_10_0.wav	       stutter_HeStutters_12_80.wav
nostutter_HeStutters_10_16.wav	       stutter_HeStutters_12_89.wav
nostutter_HeStutters_10_21.wav	       stutter_HeStutters_13_20.wav
nostutter_HeStutters_10_8.wav	       stutter_HeStutters_13_33.wav
nostutter_HeStutters_11_150.wav        stutter_HeStutters_14_27.wav
nostutter_HeStutters_11_204.wav        stutter_HeStutters_15_0.wav
nostutter_HeStutters_11_20.wav	       stutter_HeStutters_15_112.wav
nostutter_HeStutters_11_62.wav	       stutter_HeStutters_15_121.wav
nost

### Splitting to train, val, test sets

In [ ]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './sep28_balanced'

# Specify the destination directories for the split files
destination_directory_1 = './train_sep28'
destination_directory_2 = './val_sep28'
destination_directory_3 = './test_sep28'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)


1634


In [ ]:
# Specify the directory where your .wav files are located
directory1 = './train_sep28'
directory2 = './val_sep28'
directory3 = './test_sep28'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

NameError: name 'os' is not defined

In [ ]:
# import random
# import shutil
# import os
# import tensorflow as tf

# # Specify the source directory where your files are located
# source_directory = './fluency_bank_all_together'

# # Specify the destination directories for the split files
# destination_directory_1 = './fluency_balanced'
# destination_directory_2 = './fluency_inference'

# # Create the destination directories if they don't exist
# for directory in [destination_directory_1, destination_directory_2]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# # Shuffle the files in the source directory
# files = os.listdir(source_directory)
# random.shuffle(files)

# # Calculate the number of files for each split
# total_files = len(files)
# split_counts = [800, total_files-800]

# # Split the files into different lists based on the split counts
# file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]]]


# def check_elements(lst, positions):
#     non_zero_count = 0

#     for i in range(len(lst)):
#         if i in positions:
#             if i == 0 and lst[i] == 3:
#                 non_zero_count += 1
#             elif i != 0 and lst[i] >= 0:
#                 non_zero_count += 1
#         elif lst[i] != 0:
#             return False
#     return  non_zero_count >= 1

# specific_positions = [8, 9, 10, 11]

# count = 0
# # Move the files to the destination directories
# for i, file_split in enumerate(file_splits):
#     if i == 0:
#         destination_directory = destination_directory_1
#     else:
#         destination_directory = destination_directory_2

#     for filename in file_split:

#         path_parts = tf.strings.split(filename, '/')
#         path_end = path_parts[-1]
#         file_parts = tf.strings.split(path_end, '_')
#         show_name = file_parts[0].numpy().decode('utf-8')
#         pod_id = int(file_parts[-2])
#         clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
#         features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

#         if check_elements(features, specific_positions):
#             print(show_name,pod_id,clip_id)
#             print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
                


#             # source_path = os.path.join(source_directory, filename)
#             # destination_path = os.path.join(destination_directory, filename)
#             # shutil.copy2(source_path, destination_path)
    
#             count += 1

# print(count)

# 2. FLUENCY BANK PREPARATION

In [ ]:
!unzip ./fluency_bank_all_together.zip

  inflating: fluency_bank/FluencyBank_173_53.wav  
  inflating: fluency_bank/FluencyBank_173_54.wav  
  inflating: fluency_bank/FluencyBank_173_55.wav  
  inflating: fluency_bank/FluencyBank_173_56.wav  
  inflating: fluency_bank/FluencyBank_173_57.wav  
  inflating: fluency_bank/FluencyBank_173_58.wav  
  inflating: fluency_bank/FluencyBank_173_59.wav  
  inflating: fluency_bank/FluencyBank_173_6.wav  
  inflating: fluency_bank/FluencyBank_173_60.wav  
  inflating: fluency_bank/FluencyBank_173_61.wav  
  inflating: fluency_bank/FluencyBank_173_62.wav  
  inflating: fluency_bank/FluencyBank_173_63.wav  
  inflating: fluency_bank/FluencyBank_173_64.wav  
  inflating: fluency_bank/FluencyBank_173_65.wav  
  inflating: fluency_bank/FluencyBank_173_66.wav  
  inflating: fluency_bank/FluencyBank_173_67.wav  
  inflating: fluency_bank/FluencyBank_173_68.wav  
  inflating: fluency_bank/FluencyBank_173_69.wav  
  inflating: fluency_bank/FluencyBank_173_7.wav  
  inflating: fluency_bank/Fluency

In [ ]:
!ls ./fluency_bank_all_together

FluencyBank_024_111.wav  FluencyBank_096_84.wav   FluencyBank_173_56.wav
FluencyBank_024_112.wav  FluencyBank_096_85.wav   FluencyBank_173_57.wav
FluencyBank_024_113.wav  FluencyBank_096_86.wav   FluencyBank_173_58.wav
FluencyBank_024_114.wav  FluencyBank_096_87.wav   FluencyBank_173_59.wav
FluencyBank_024_115.wav  FluencyBank_096_88.wav   FluencyBank_173_5.wav
FluencyBank_024_116.wav  FluencyBank_096_89.wav   FluencyBank_173_60.wav
FluencyBank_024_117.wav  FluencyBank_096_8.wav	  FluencyBank_173_61.wav
FluencyBank_024_118.wav  FluencyBank_096_90.wav   FluencyBank_173_62.wav
FluencyBank_024_119.wav  FluencyBank_096_91.wav   FluencyBank_173_63.wav
FluencyBank_024_11.wav	 FluencyBank_096_92.wav   FluencyBank_173_64.wav
FluencyBank_024_120.wav  FluencyBank_096_93.wav   FluencyBank_173_65.wav
FluencyBank_024_121.wav  FluencyBank_096_94.wav   FluencyBank_173_66.wav
FluencyBank_024_122.wav  FluencyBank_096_9.wav	  FluencyBank_173_67.wav
FluencyBank_024_123.wav  FluencyBank_097_0.wav	  Fluenc

In [ ]:
import os

root_folder = './fluency_bank_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 4144


In [1]:
import pandas as pd

df_flu = pd.read_csv('fluencybank_labels.csv')
df_flu.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,FluencyBank,10,0,88960,136960,0,0,0,1,0,0,0,0,2,0,0,0
1,FluencyBank,10,1,1271520,1319520,0,0,0,0,0,0,0,0,3,0,0,0
2,FluencyBank,10,2,1813760,1861760,0,0,1,0,0,0,0,0,2,0,0,0
3,FluencyBank,10,3,1842720,1890720,0,0,1,0,0,0,0,0,2,1,0,0
4,FluencyBank,10,4,1893280,1941280,0,0,0,0,0,3,0,0,0,0,0,0


In [ ]:
column_location = df_flu.columns.get_loc('NoStutteredWords')
loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

loc_NoStutteredWords,loc_NaturalPause,loc_Music,loc_NoSpeech

(13, 14, 15, 16)

In [ ]:
listttt = df_flu[(df_flu['Show'] == 'FluencyBank') & (df_flu['EpId'] == 10) & (df_flu['ClipId'] == 0)].to_numpy()[0]
ffff = [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0]

loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

check_elements(ffff, specific_positions), listttt[5:]

(True, array([0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0], dtype=object))

### Creating no stutter files with confidence 3 and count of 800 including 'NoStutteredWords', 'NoSpeech',  'Music' and 'NaturalPause'

In [8]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 0) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] == 0)
                        & (df_flu['Block'] == 0)
                        & (df_flu['SoundRep'] == 0)    
                        & (df_flu['WordRep'] >0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] == 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 38)

In [ ]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_balanced'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

[0 0 0 0 0 0 0 0 3 1 0 0]     NoStutteredWords  NaturalPause  Music  NoSpeech
36                 3             1      0         0
FluencyBank 24 35
[0 0 0 0 0 0 0 0 3 1 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
405                 3             1      0         0
FluencyBank 19 42
[0 0 0 0 0 0 0 0 3 1 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
323                 3             1      0         0
FluencyBank 219 92
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3736                 3             0      0         0
FluencyBank 99 22
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
2121                 3             0      0         0
FluencyBank 98 54
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
2025                 3             0      0         0
FluencyBank 111 61
[0 0 0 0 0 0 0 0 3 1 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
2528          

In [ ]:
import os

root_folder = './fluency_balanced/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 931


### Creating stutter files for  'Prolongation', 'SoundRep', 'WordRep', 'Block',  'Interjection' to be detected as stuttering situation and count of 923

In [ ]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 0) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] >= 0)
                        & (df_flu['Block'] >= 0)
                        & (df_flu['SoundRep'] >= 0)    
                        & (df_flu['WordRep'] >= 0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] >= 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 923)

In [ ]:
loc_Prolongation = df_flu.columns.get_loc('Prolongation')
loc_Block = df_flu.columns.get_loc('Block')
loc_SoundRep = df_flu.columns.get_loc('SoundRep')
loc_WordRep = df_flu.columns.get_loc('WordRep')
loc_Interjection = df_flu.columns.get_loc('Interjection')

loc_Prolongation, loc_Block, loc_SoundRep, loc_WordRep, loc_Interjection

(7, 8, 9, 10, 12)

In [ ]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_balanced'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if lst[i] != 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [2, 3, 4, 5, 7]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['Prolongation', 'Block','SoundRep','WordRep','Interjection']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

[0 0 0 1 2 0 0 0 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
3797             0         2        0             0
FluencyBank 96 78
[0 0 1 1 1 0 0 0 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
1761             1         1        0             0
FluencyBank 230 142
[0 0 0 1 1 2 0 0 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
3912             0         1        2             0
FluencyBank 109 55
[0 0 0 1 2 0 0 3 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
2437             0         2        0             3
FluencyBank 19 15
[0 0 2 0 0 1 0 1 0 0 0 0]      Prolongation  SoundRep  WordRep  Interjection
296             2         0        1             1
FluencyBank 87 40
[0 0 0 1 3 0 0 2 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
1373             0         3        0             2
FluencyBank 96 34
[0 0 1 1 2 0 0 0 0 0 0 0]       Prolongation  SoundRep  WordRep  Interjection
1717             1         2     

In [ ]:
import os

root_folder = './fluency_balanced/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 1854


### Renaming files in fluency_balanced to nostutter (931) and stutter files(923):

In [2]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './fluency_balanced_2/'


def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if check_elements(features, specific_positions):
        print('Nooooooooooooo',features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)
        

    else:
        print('YYYYeeeeesssssss',features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

Nooooooooooooo [0 0 0 0 0 0 0 0 3 2 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 2 0 0]
YYYYeeeeesssssss [0 0 1 2 3 0 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
YYYYeeeeesssssss [0 0 0 2 2 0 0 0 0 0 0 0]
YYYYeeeeesssssss [0 0 0 1 2 0 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
YYYYeeeeesssssss [0 0 0 2 0 1 0 0 0 0 0 0]
YYYYeeeeesssssss [0 0 0 0 0 3 0 1 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
YYYYeeeeesssssss [0 0 0 0 3 1 0 2 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
YYYYeeeeesssssss [0 0 0 1 2 0 0 3 0 0 0 0]
YYYYeeeeesssssss [0 0 0 1 2 0 0 3 0 0 0 0]
YYYYeeeeesssssss [0 0 2 1 2 3 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0

In [3]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_balanced/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 931 , 923 and tot 1854


In [4]:
!ls ./fluency_balanced

nostutter_FluencyBank_010_10.wav   nostutter_FluencyBank_985_87.wav
nostutter_FluencyBank_010_13.wav   nostutter_FluencyBank_985_94.wav
nostutter_FluencyBank_010_14.wav   nostutter_FluencyBank_985_95.wav
nostutter_FluencyBank_010_16.wav   nostutter_FluencyBank_985_98.wav
nostutter_FluencyBank_010_1.wav    stutter_FluencyBank_010_38.wav
nostutter_FluencyBank_010_20.wav   stutter_FluencyBank_010_4.wav
nostutter_FluencyBank_010_21.wav   stutter_FluencyBank_010_75.wav
nostutter_FluencyBank_010_22.wav   stutter_FluencyBank_010_79.wav
nostutter_FluencyBank_010_23.wav   stutter_FluencyBank_016_103.wav
nostutter_FluencyBank_010_25.wav   stutter_FluencyBank_016_105.wav
nostutter_FluencyBank_010_26.wav   stutter_FluencyBank_016_112.wav
nostutter_FluencyBank_010_27.wav   stutter_FluencyBank_016_13.wav
nostutter_FluencyBank_010_30.wav   stutter_FluencyBank_016_17.wav
nostutter_FluencyBank_010_31.wav   stutter_FluencyBank_016_26.wav
nostutter_FluencyBank_010_32.wav   stutter_FluencyBank_016_31.wav


### Splitting to train, val, test sets

In [5]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './fluency_balanced'

# Specify the destination directories for the split files
destination_directory_1 = './train_fluency'
destination_directory_2 = './val_fluency'
destination_directory_3 = './test_fluency'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)

1853


In [6]:
# Specify the directory where your .wav files are located
directory1 = './train_fluency'
directory2 = './val_fluency'
directory3 = './test_fluency'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of nostutter files in the train folder: 736 and tot is 1483
Number of nostutter files in the val folder: 101 and tot is 185
Number of nostutter files in the test folder: 93 and tot is 185


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>